In [ ]:
!unzip ./data/song-data.zip

In [1]:
!unzip ./data/log-data.zip -d ./data/log_data

Archive:  ./data/log-data.zip
  inflating: ./data/log_data/2018-11-01-events.json  
  inflating: ./data/log_data/2018-11-02-events.json  
  inflating: ./data/log_data/2018-11-03-events.json  
  inflating: ./data/log_data/2018-11-04-events.json  
  inflating: ./data/log_data/2018-11-05-events.json  
  inflating: ./data/log_data/2018-11-06-events.json  
  inflating: ./data/log_data/2018-11-07-events.json  
  inflating: ./data/log_data/2018-11-08-events.json  
  inflating: ./data/log_data/2018-11-09-events.json  
  inflating: ./data/log_data/2018-11-10-events.json  
  inflating: ./data/log_data/2018-11-11-events.json  
  inflating: ./data/log_data/2018-11-12-events.json  
  inflating: ./data/log_data/2018-11-13-events.json  
  inflating: ./data/log_data/2018-11-14-events.json  
  inflating: ./data/log_data/2018-11-15-events.json  
  inflating: ./data/log_data/2018-11-16-events.json  
  inflating: ./data/log_data/2018-11-17-events.json  
  inflating: ./data/log_data/2018-11-18-events.json 

In [24]:
!rm -rf ./data/parquet

In [2]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import (year, month, dayofmonth, hour, weekofyear, 
                                    date_format, dayofweek, monotonically_increasing_id)

In [3]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [4]:
spark = create_spark_session()

In [5]:
input_data = "./data/"
output_data = "./data/parquet"

In [6]:
def process_song_data(spark, input_data, output_data):
    # get filepath to song data file
    song_data = input_data + 'song_data/*/*/*/*.json'
    
    # read song data file
    df = spark.read.json(song_data)

    # extract columns to create songs table
    songs_table = df.select(['song_id','title','artist_id','year','duration']) \
                    .dropDuplicates()
    
    # write songs table to parquet files partitioned by year and artist
    songs_table.write.partitionBy(['year', 'artist_id']) \
                .parquet("{}/parquet/songs".format(output_data), mode="overwrite")

    # extract columns to create artists table
    artists_table = df.select(['artist_id', 'artist_name', 'artist_location',
                              'artist_latitude', 'artist_longitude']) \
                        .withColumnRenamed('artist_name', 'name') \
                        .withColumnRenamed('artist_location', 'location') \
                        .withColumnRenamed('artist_latitude', 'latitude') \
                        .withColumnRenamed('artist_longitude', 'longitude') \
                        .dropDuplicates() 
    
    # write artists table to parquet files
    artists_table.write.parquet("{}/parquet/artists".format(output_data), mode='overwrite')

In [13]:
def process_log_data(spark, input_data, output_data):
    # get filepath to log data file
    log_data = input_data + 'log_data'

    # read log data file
    df = spark.read.json(log_data)
    
    # filter by actions for song plays
    events_df = df.filter(df.page == 'NextSong') \
                   .select(['ts', 'userId', 'level', 'song', 'artist',
                           'length', 'sessionId', 'location', 'userAgent'])

    # extract columns for users table    
    users_table = df.select(['userId', 'firstName', 'lastName',
                            'gender', 'level']).dropDuplicates()
    
    # write users table to parquet files
    users_table.write.parquet("{}/parquet/users".format(output_data), mode='overwrite')

    # create timestamp column from original timestamp column
    get_timestamp = udf(lambda x: str(int(int(x)/1000)))
    events_df = events_df.withColumn('timestamp', get_timestamp(events_df.ts)) 
    
    # create datetime column from original timestamp column
    get_datetime = udf(lambda x: str(datetime.fromtimestamp(int(x) / 1000)))
    events_df = events_df.withColumn('datetime', get_datetime(events_df.ts)) \
                        .withColumnRenamed('datetime', 'start_time')
    
    # extract columns to create time table
    time_table = events_df.select('start_time') \
                           .withColumn('hour', hour('start_time')) \
                           .withColumn('day', dayofmonth('start_time')) \
                           .withColumn('week', weekofyear('start_time')) \
                           .withColumn('month', month('start_time')) \
                           .withColumn('year', year('start_time')) \
                           .withColumn('weekday', dayofweek('start_time')) \
                           .dropDuplicates()
    
    # write time table to parquet files partitioned by year and month
    time_table.write.partitionBy('year', 'month') \
                    .parquet("{}/parquet/time".format(output_data), 'overwrite')

    # read in song data to use for songplays table
    stg_songs_df = spark.read.json(input_data + 'song_data/*/*/*/*.json')

    # extract columns from joined song and log datasets to create songplays table 
    joined_df = events_df.join(stg_songs_df, \
                        (events_df.artist == stg_songs_df.artist_name) \
                        & (events_df.song == stg_songs_df.title) \
                        & (events_df.length == stg_songs_df.duration), \
                    how='inner')

    songplays_table = joined_df.select(['start_time', 'userId', 'level', 'song_id', \
                                    'artist_id', 'sessionId', 'artist_location', 'userAgent']) \
                                .withColumnRenamed('userId', 'user_id') \
                                .withColumnRenamed('sessionId', 'session_id') \
                                .withColumnRenamed('artist_location', 'location') \
                                .withColumnRenamed('userAgent', 'user_agent') \
                                .dropDuplicates() \
                                .withColumn('year', year('start_time')) \
                                .withColumn('month', month('start_time')) \
                                .withColumn('songplay_id', monotonically_increasing_id())

    # write songplays table to parquet files partitioned by year and month
    songplays_table.write.partitionBy(['year', 'month']) \
            .parquet("{}/parquet/songplays".format(output_data), mode="overwrite")

In [12]:
# process_song_data(spark, input_data, output_data)   
# print('\nDone 1\n')
process_log_data(spark, input_data, output_data)
print('\nDone 2\n')


Done 2



In [17]:
import glob

In [23]:
log_data = input_data + 'song_data/**/*.json'
log_data = glob.glob("data/song_data/**/*.json", recursive=True)
df = spark.read.json(log_data)

df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



# Data Lake with Spark and S3

## Introduction
A music streaming startup, Sparkify, has grown their user base and song database and want to move their processes and data onto the cloud. This project is responsible for building an ETL pipeline in order to their analytics team to continue finding insights into what songs their users are listening to. The ETL pipeline inlcude:
- Extracts their data from S3
- Processes them using Spark
- Transforms data into S3 as a set of dimensional tables

## Data Warehouse schema design
- Fact table: songplays
- Dimension tables: users, songs, artists, time 

## How to run the Python Scripts
Fistly, you need to copy the AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY, paste them into `dl.cfg`.
After that, run the command
```
    python etl.py
```

## Explanation
- `dl.cfg` is where contains your AWS credentials
- `etl.py` is where you'll reads data from S3, processes that data using Spark, and writes them back to S3.
